<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gan/colab_gan_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2021 by [Jeff Heaton](https://www.youtube.com/channel/UCR1-GEpyOPzT2AO4D_eifdw), [released under Apache 2.0 license](https://github.com/jeffheaton/present/blob/master/LICENSE)


# Training StyleGAN2 in Google CoLab  

### Google Drive setups

One of the main requirements for starting is google drive. We're going to use G-DRIVE to store our training data and trained neural networks. we built some folders with the path of follows:
```
content/drive/MyDrive/dogs/data/gan 
``` 
in gan's folder, we have three other folders, "images" for raw cropped images we want to train on. The "datasets" folder, the Actual training datasets that I convert from the raw images, and the "experiments" folder includes the output from StyleGAN2, the image previews, and saved network snapshots.

### Managing GPU

There are two options for training GANs with Google Colab, Google Colab Free or Pro. We will go with the pro one because it has advantages like better GPU, longer runtime, and timeouts, and most important, it will not disconnect before 24 hours. In using the free version of google colab, you should make sure that you run the notebook with GPU runtime. For doing this, go to "Runtime" in the bar and select the "change the runtime type" option. Then, put the Hardware accelerator on GPU and save it.

In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


### Steps of the code

1) connecting Colab to G-DRIVE, to save the snapshots there

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


3) creating needful directories in GDRIVE

In [ ]:
!mkdir drive/MyDrive/dogs
!mkdir drive/MyDrive/dogs/data
!mkdir drive/MyDrive/dogs/data/gan
!mkdir drive/MyDrive/dogs/data/gan/images
!mkdir drive/MyDrive/dogs/data/gan/dataset
!mkdir drive/MyDrive/dogs/data/gan/experiments

3) unzipping the training images in "images" folder

In [ ]:
!unzip drive/MyDrive/images.zip -d drive/MyDrive/dogs/data/gan/images

4) Number of total training images and establishig the exact path of images

In [ ]:
!ls drive/MyDrive/dogs/data/gan/images | wc -l

5) Installing NVIDIA StyleGAN2 ADA PyTorch

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

6) This code takes a python file and converts the first given path to tensors into the second given direction. We can face an error in this part in conditions like inconsistent types and variant sizes of images. We convert all the images into jpg type and resize them to 128 x 128 pixels.

!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/data/gan/images/circuit --dest /content/drive/MyDrive/data/gan/dataset/circuit

7) setting the version of the torch, torchvision,  torchaudio to become compatible with Style-GAN2ada

In [5]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

^C


8) to clear out the newly created dataset in case something went wrong 

In [ ]:
!rm -rf /root/.cache/torch_extensions/*

9) making all the images to the exact dimensions and color depth

In [ ]:

import os
from tqdm.notebook import tqdm
from PIL import Image
from os import listdir

IMAGE_PATH = 'drive/MyDrive/dogs/data/gan/images'
files = [f for f in listdir(IMAGE_PATH) if os.path.isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
    file2 = os.path.join(IMAGE_PATH,file)
    img = Image.open(file2)
    sz = img.size
    if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
    elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
    else:
    base_size = sz


10) Converting all the images to jpg type

In [ ]:
from PIL import Image
import os, sys

path = "drive/MyDrive/dogs/data/gan/images"

for item in os.listdir(path):
    im = Image.open(path + '/' +  item)
    if im.mode != "RGB" :
        im = im.convert("RGB")
    imResize = im.resize((128,128))
    imResize.save(path + '/' +  item )



### Perform Initial Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/dogs/data/gan/experiments"
DATA = "/content/drive/MyDrive/dogs/data/gan/dataset"
SNAP = 20

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

### Resume Training

11) removing the last trained network into the experiment folder for continuing the training

In [ ]:
!rm drive/MyDrive/dogs/data/gan/experiments/network-snapshot* drive/MyDrive/dogs/data/gan/experiments

In [ ]:
import os

EXPERIMENTS = "/content/drive/MyDrive/dogs/data/gan/experiments/"
NETWORK = "network-snapshot-000480.pkl"
RESUME = os.path.join(EXPERIMENTS, NETWORK)
DATA = "/content/drive/MyDrive/dogs/data/gan/dataset"
SNAP = 20

cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

# Find Your Files

The drive is mounted to the following location.

```
/content/drive/MyDrive/data
```

It might be helpful to use an ```ls``` command to establish the exact path for your images.

In [ ]:
!ls /content/drive/MyDrive/data/gan/images

# Convert Your Images

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/data/gan/images/circuit --dest /content/drive/MyDrive/data/gan/dataset/circuit

The following command can be used to clear out the newly created dataset.  If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [ ]:
#!rm -R /content/drive/MyDrive/data/gan/dataset/circuit/*

In [ ]:
Perform Initial Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/dogs/data/gan/experiments"
DATA = "/content/drive/MyDrive/dogs/data/gan/dataset"
SNAP = 20

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

In [ ]:
!rm drive/MyDrive/dogs/data/gan/experiments/network-snapshot*

In [ ]:
!mv drive/MyDrive/network-snapshot-000480.pkl drive/MyDrive/dogs/data/gan/experiments

# Clean Up your Images

It is important that all images have the same dimensions and color depth.  This code can identify images that have issues.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images/fish'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


# Perform Initial Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset/circuit"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

In [ ]:
!/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap 25 --resume /content/drive/MyDrive/data/gan/experiments/00007-circuit-auto1/network-snapshot-000500.pkl --outdir /content/drive/MyDrive/data/gan/experiments --data /content/drive/MyDrive/data/gan/dataset/circuit

# Resume Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000100.pkl"
RESUME = os.path.join(EXPERIMENTS, "00008-circuit-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset/circuit"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}